In [ ]:
import pandas as pd
import numpy as np

In [ ]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  COLLAB = True
else:
  COLLAB = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if COLLAB == True:
  df = pd.read_csv('/content/drive/Shareddrives/ETICA/dataset_newtral_ucm.csv')
else:
  df = pd.read_csv('../data/dataset_newtral_ucm.csv')

In [ ]:
# !pip install transformers

In [ ]:
import transformers
from transformers import pipeline
pipe = pipeline("text-classification", model="Newtral/xlm-r-finetuned-toxic-political-tweets-es")

In [ ]:
lideres = ["Casado", "Ayuso", "Abascal", "Monasterio" ,"Sanchez", "Sánchez", "Iglesias", "Yolanda", "Arrimadas", "Rivera"]
ids = ["@pablocasado", "@IdiazAyuso", "@Santi_ABASCAL", "@monasterioR", "@sanchezcastejon", "@Yolanda_Diaz", "@InesArrimadas"] #Pablo iglesias y Albert Rivera salen demasiado poco sus @
nombresLideres  = set(lideres).union(set(ids))
nombresLideres

{'@IdiazAyuso',
 '@InesArrimadas',
 '@Santi_ABASCAL',
 '@Yolanda_Diaz',
 '@monasterioR',
 '@pablocasado',
 '@sanchezcastejon',
 'Abascal',
 'Arrimadas',
 'Ayuso',
 'Casado',
 'Iglesias',
 'Monasterio',
 'Rivera',
 'Sanchez',
 'Sánchez',
 'Yolanda'}

In [ ]:
df.head(5)

,twitter_id,text,toxicity,sev_toxicity,profanity,threat,degree1,degree2,degree3,degree_final,degree_predicted,degree_confidence,annotation_date,author_id,slug,twitter_url,party_slug,parliamentary_group_slug
0,1344806255798480896,Feliz 2021 #Adios2020 https://t.co/ULpRE0T2AI,0.012811,0.022777,0.010508,0.077519,0,0,0,0,0,0.969,2021-10-07,2849798849,jose-alberto-herrero-bono,https://twitter.com/herrerobono,partido-popular,gp
1,1344822586396602368,Empezar el año con #CachitosNochevieja no tien...,0.085995,0.068680,0.302721,0.087808,0,0,0,0,0,0.984,2021-10-07,88802757,begona-nasarre-oliva,https://twitter.com/Begonasarre,psoe,gs
2,1344934218145669120,Llegará un momento que los miembros de este Go...,0.478526,0.122059,0.026271,0.513052,2,2,1,2,2,0.868,2021-10-07,593952938,juan-luis-steegmann-olmedillas,https://twitter.com/jlsteeg,vox,gvox
3,1344940715072221184,La frase “ Falsa sensación de seguridad” viene...,0.309307,0.040751,0.009823,0.122409,2,1,1,1,2,0.557,2021-08-25,593952938,juan-luis-steegmann-olmedillas,https://twitter.com/jlsteeg,vox,gvox
4,1344945712128266240,El Comisario político del Gobierno no descansa...,0.194637,0.072107,0.026215,0.249154,2,2,2,2,2,0.964,2021-10-07,1197128623364423680,macarena-montesinos-de-miguel,https://twitter.com/MackMontesinos,partido-popular,gp


In [ ]:
column_names = ["twitter_id", "original_text", "slug", "party_slug", "original_name", "toxic_original", "very_toxic_original", "most_toxic_leader", "most_toxic_leader_toxicity", "most_verytoxic_leader" ,"most_verytoxic_leader_toxicity", "LIDER_toxic", "LIDER_verytoxic"]
df_wordmod = pd.DataFrame(columns = column_names)
i = 0
for idx, row in df.iterrows():
  for word in row.text.split():
    if(word in nombresLideres):
      row_tox_value = pipe(row.text, return_all_scores=True)[0][0]['score']
      row_vtox_value = pipe(row.text, return_all_scores=True)[0][1]['score']
      maxt = row_tox_value
      maxtp = word
      maxvt = row_vtox_value
      maxvtp = word
      for otro in nombresLideres:
        if word != otro:
          value = pipe(row.text.replace(word, otro), return_all_scores=True)[0]
          # if pipe(row.text.replace(word, otro), return_all_scores=True)[0][0]['score'] > maxt:
          if (value[0]['score'] > maxt):
            # maxt = pipe(row.text.replace(word, otro), return_all_scores=True)[0][0]['score']
            maxt = value[0]['score']
            maxtp = otro
          # if pipe(row.text.replace(word, otro), return_all_scores=True)[0][1]['score'] > maxvt:
          if (value[1]['score'] > maxvt):
            # maxvt = pipe(row.text.replace(word, otro), return_all_scores=True)[0][1]['score']
            maxvt = value[1]['score']
            maxvtp = otro
      neutral_word = pipe(row.text.replace(word, "partido"), return_all_scores=True)[0]
      new_row = {'twitter_id':row.twitter_id, 'original_text':row.text, 'slug':row.slug, 'party_slug':row.party_slug, 'original_name': word, 'toxic_original':row_tox_value , 'very_toxic_original':row_vtox_value,'most_toxic_leader':maxtp, 'most_toxic_leader_toxicity': maxt, 'most_verytoxic_leader':maxvtp, 'most_verytoxic_leader_toxicity': maxvt, 'LIDER_toxic': neutral_word[0]['score'],'LIDER_verytoxic':neutral_word[1]['score']}
      df_wordmod = df_wordmod.append(new_row, ignore_index = True)
      i+=1

In [ ]:
df_wordmod

,twitter_id,original_text,slug,party_slug,original_name,toxic_original,very_toxic_original,most_toxic_leader,most_toxic_leader_toxicity,most_verytoxic_leader,most_verytoxic_leader_toxicity,LIDER_toxic,LIDER_verytoxic
0,1345429153357701121,El acuerdo del Gobierno con Reino Unido sobre ...,concepcion-gamarra-ruiz-clavijo,partido-popular,Sánchez,0.004959,0.002321,@Santi_ABASCAL,0.005231,@monasterioR,0.002508,0.006789,0.001784
1,1345630564498481153,Los españoles hartos de la irresponsabilidad d...,carmen-riolobos-regadera,partido-popular,Iglesias,0.993780,0.109731,Monasterio,0.994659,Monasterio,0.188398,0.993590,0.099156
2,1345650372191379456,Casado se sitúa a menos de tres puntos de Sánc...,ana-belen-vazquez-blanco,partido-popular,Casado,0.032812,0.000795,Monasterio,0.236452,Ayuso,0.001158,0.060692,0.000806
3,1345650372191379456,Casado se sitúa a menos de tres puntos de Sánc...,ana-belen-vazquez-blanco,partido-popular,Sánchez,0.032812,0.000795,Yolanda,0.209637,Ayuso,0.000876,0.029122,0.000851
4,1345690120033349645,¿ Qué está pasando?\n\n➡️ Que los españoles ya...,javier-merino-martinez,partido-popular,Sanchez,0.316567,0.000900,Monasterio,0.778970,Monasterio,0.002471,0.273959,0.000843
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,1451844662113755142,El Gobierno de @sanchezcastejon que dice que l...,eloy-suarez-lamata,partido-popular,@sanchezcastejon,0.010879,0.001078,Iglesias,0.953319,Iglesias,0.005580,0.096544,0.000705
1546,1453301708390547457,"Sánchez y la mentira van siempre de la mano, e...",jaime-eduardo-de-olano-vela,partido-popular,Sánchez,0.995358,0.996276,Casado,0.996924,Sánchez,0.996276,0.996774,0.981104
1547,1453339175256743940,La vivienda es sobre todo un derecho humano y ...,alfonso-rodriguez-gomez-de-celis,psoe,@sanchezcastejon,0.004295,0.002777,Iglesias,0.010003,Abascal,0.003100,0.004707,0.002406
1548,1451049912062386184,67 días desde que empezó el #IFNavalacruz en #...,alicia-garcia-rodriguez,partido-popular,Sánchez,0.006563,0.002006,Iglesias,0.008175,@IdiazAyuso,0.002317,0.010167,0.001447


In [ ]:
if COLLAB == True:
  df_wordmod.to_csv('/content/drive/Shareddrives/ETICA/tweets_mods_politicians.csv', index=False)  
else:
  df_wordmod.to_csv('../data/tweets_mods_politicians.csv', index=False)